In [5]:
import numpy as np
from PIL import Image

def mi_convolucion(imagen, kernel, padding_type='reflect'):
    if imagen.ndim != 2:
        raise ValueError("imagen en formato incorrecto")
    
    imagen = imagen.astype(np.float64)
    kernel = kernel.astype(np.float64)
    
    kernel_flipped = np.flip(kernel)
    
    kernel_h, kernel_w = kernel.shape
    pad_h = kernel_h // 2
    pad_w = kernel_w // 2
    
    imagen_padded = np.pad(
        imagen,
        pad_width=((pad_h, pad_h), (pad_w, pad_w)),
        mode=padding_type
    )
    
    altura, anchura = imagen.shape
    resultado = np.zeros_like(imagen)
    
    for i in range(kernel_h):
        for j in range(kernel_w):
            vista = imagen_padded[i:i+altura, j:j+anchura]
            resultado += vista * kernel_flipped[i, j]
    
    return resultado



imagen = np.array(Image.open('foto.jpg').convert('L'))

kernel_blur = np.ones((3, 3)) / 9

resultado = mi_convolucion(imagen, kernel_blur)


resultado_img = Image.fromarray(resultado.astype(np.uint8))
resultado_img.save('resultado.jpg')

In [6]:

def generar_gaussiano(tamano, sigma):
    centro = tamano // 2
    x = np.arange(tamano) - centro
    y = np.arange(tamano) - centro
    
    xx, yy = np.meshgrid(x, y)

    gaussiano = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    
    gaussiano = gaussiano / np.sum(gaussiano)
    
    return gaussiano



kernel = generar_gaussiano(5, sigma=1.0)

print("Kernel 5x5")
print(kernel)
print(f"\nSuma total: {np.sum(kernel)}")  

Kernel 5x5
[[0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.02193823 0.09832033 0.16210282 0.09832033 0.02193823]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]]

Suma total: 1.0


In [7]:

def detectar_bordes_sobel(imagen):

    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float64)
    
    sobel_y = np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float64)
    
    Gx = mi_convolucion(imagen, sobel_x)
    Gy = mi_convolucion(imagen, sobel_y)
    
    magnitud = np.sqrt(Gx**2 + Gy**2)
    
    magnitud = (magnitud / magnitud.max()) * 255
    
    direccion = np.arctan2(Gy, Gx)  
    
    return magnitud, direccion


from PIL import Image

imagen = np.array(Image.open('foto.jpg').convert('L'))

magnitud, direccion = detectar_bordes_sobel(imagen)

print(f"Magnitud - min: {magnitud.min():.2f}, max: {magnitud.max():.2f}")
print(f"Dirección - min: {direccion.min():.2f}, max: {direccion.max():.2f} radianes")

resultado_img = Image.fromarray(magnitud.astype(np.uint8))
resultado_img.save('bordes.jpg')

Magnitud - min: 0.00, max: 255.00
Dirección - min: -3.14, max: 3.14 radianes
